# Liquidity Mining Program

Global imports

In [52]:
import requests
url = 'https://vanir.s.singularity.so/v1/graphql'
headers = {
  'Content-Type' : 'application/json',
  'x-hasura-admin-secret' : 'password@2021'
}


Utilities

In [60]:
def executeQuery(query_json):
  r = requests.post(url=url, json=query_json, headers=headers).json()
  return r['data']

def getDecimalDict():
  token_list_url = 'https://raw.githubusercontent.com/Sifchain/sifnode/613ddd317c307a6e12e4f3ddf87000f4e40b31e6/ui/core/src/assets.sifchain.mainnet.json'
  token_list = requests.get(token_list_url).json()
  token_decimals_dictionary = {'ceth':18}
  for token in token_list['assets']:
    try:
        token_decimals_dictionary[token['symbol'].lower()] = token['decimals']
    except KeyError as e:
        pass
  return token_decimals_dictionary

def getPools():
  url = 'https://api.sifchain.finance/clp/getPools'
  r = requests.get(url).json()
  pools = r['result']['Pools']
  token_decimals_dictionary = getDecimalDict()
  pool_data ={}
  for i in pools:
    d=i
    symbol = i['external_asset']['symbol']
    d['decimals'] = token_decimals_dictionary[symbol]
    pool_data[symbol] = d
  return pool_data

def calculatePoolShare(address):
  al_query_str = '''
  query GetLiquidityProvidersWithAmounts { events_betanet(
    where: {al_provider: {_eq: \"%s\"}}) 
    { al_provider al_amount al_token timestamp } }
    ''' % address

  al_query_json = {"query": al_query_str}
  al_json = executeQuery(al_query_json)

  rl_query_str = '''
  query GetLiquidityProvidersWithAmounts { events_betanet(
    where: {rl_provider: {_eq: \"%s\"}}) 
    { rl_provider rl_amount rl_token timestamp } }
    ''' % address

  rl_query_json = {"query": rl_query_str}
  rl_json = executeQuery(rl_query_json)

  pool_share = {}

  for i in al_json['events_betanet']:
    pool_share[i['al_token']] = 0

  for i in rl_json['events_betanet']:
    pool_share[i['rl_token']] = 0
  
  for i in al_json['events_betanet']:
    pool_share[i['al_token']] += i['al_amount']

  for i in rl_json['events_betanet']:
    pool_share[i['rl_token']] -= i['rl_amount']

  pools = getPools()
  pool_data=[]
  for token,amount in pool_share.items():
    d={}
    d['symbol'] = token
    d['amount'] = amount
    d['share'] = 0
    pool_data.append(d)
  
  for i in pool_data:
    nab = pools[i['symbol']]['native_asset_balance']
    decimals = pools[i['symbol']]['decimals']
    normalised_nab = float(nab)/10**(decimals)
    share = i['amount']/normalised_nab
    i['share'] = share
    
  return pool_data

#calculatePoolShare('sif1sk4jsedru5ezt47hy52z3tkpts9m04henfrema')


[{'amount': -77.7987231148,
  'rewards': -1.8747878030938067e-11,
  'share': -2.2497462636110735e-17,
  'symbol': 'cusdt'},
 {'amount': 0.0, 'rewards': 0.0, 'share': 0.0, 'symbol': 'cusdc'}]

Get all validators at current block height




In [ ]:
getValidatorQuery = {"query": "query GetValidators { validators_betanet(limit: 1 order_by: {height: desc}) { validators } }"}

r = requests.post(url=url, json=getValidatorQuery, headers=headers).json()
print(r['data']['validators_betanet'][0]['validators'])

['031476B028E64ACFB853C071EA2E3F8FFDD0E339', '382A9BC51EA9DBEB0C1544CCFFD966F8CFFD6042', '521D35251D583A10C026CB25D38C9AA2DE3DB0F6', '6B0F1613D678B79106C203BC02D1775F92874A80', '74CBE9A6993BE6FEB07E8EBB40C600721259A34D', '988B9BDFA676A1E2B298AEBAADA3C08F6A156EC7', 'B692AE163794E65B5DD44FE4587749625F460921', 'EBB7E5425BEF921B85FB5C138D404443E94BAAC7']


Get list of all Liquidity Providers

In [ ]:
getLiquidityProviderQuery = {"query": "query GetLiquidityProviders { events_betanet(where: {type: {_eq: \"added_liquidity\"} timestamp: {}}) { al_provider } }"}

r = requests.post(url=url, json=getLiquidityProviderQuery, headers=headers).json()
al_providers = r['data']['events_betanet']
lp_arr = [i['al_provider'] for i in al_providers]
print(lp_arr)

['sif13sdapj5h0xk4rpns0sv5zgpnzmnj6ssymr6d32', 'sif1hjkgsq0wcmwdh8pr3snhswx5xyy4zpgs833akh', 'sif1k3ldrhrtkr6ncgjlu2ltyls9rxqtu9643lknrk', 'sif1hjkgsq0wcmwdh8pr3snhswx5xyy4zpgs833akh', 'sif1syavy2npfyt9tcncdtsdzf7kny9lh777yqc2nd', 'sif13rhur8wxxkgw4v9ktpqke48wsycm3cznn58am8', 'sif1hjkgsq0wcmwdh8pr3snhswx5xyy4zpgs833akh', 'sif1hjkgsq0wcmwdh8pr3snhswx5xyy4zpgs833akh']


Get Liquidity Providers with amounts

In [ ]:
getLiquidityProviderWithAmountsQuery= {"query": "query GetLiquidityProvidersWithAmounts { events_betanet(where: {type: {_eq: \"added_liquidity\"} timestamp: {}}) { al_provider al_amount al_token } }"}
r = requests.post(url=url, json=getLiquidityProviderQuery, headers=headers).json()
al_providers = r['data']['events_betanet']

print(r)

{'data': {'events_betanet': [{'al_provider': 'sif13sdapj5h0xk4rpns0sv5zgpnzmnj6ssymr6d32'}, {'al_provider': 'sif1hjkgsq0wcmwdh8pr3snhswx5xyy4zpgs833akh'}, {'al_provider': 'sif1k3ldrhrtkr6ncgjlu2ltyls9rxqtu9643lknrk'}, {'al_provider': 'sif1hjkgsq0wcmwdh8pr3snhswx5xyy4zpgs833akh'}, {'al_provider': 'sif1syavy2npfyt9tcncdtsdzf7kny9lh777yqc2nd'}, {'al_provider': 'sif13rhur8wxxkgw4v9ktpqke48wsycm3cznn58am8'}, {'al_provider': 'sif1hjkgsq0wcmwdh8pr3snhswx5xyy4zpgs833akh'}, {'al_provider': 'sif1hjkgsq0wcmwdh8pr3snhswx5xyy4zpgs833akh'}]}}


Get pool share 

In [61]:
#vesting multiplier (VM) Liquidity Lock Multiplier (LLM)
def calculateRewards(address, vm=1, llm=1):
  rowans = 833333
  pool_share = calculatePoolShare(address)
  rewards = []
  for i in pool_share:
    d=i
    d['rewards'] = rowans * i['share'] * vm * llm
    rewards.append(d)

  return rewards

calculateRewards('sif1sk4jsedru5ezt47hy52z3tkpts9m04henfrema')

[{'amount': -77.7987231148,
  'rewards': -1.8747878030938067e-11,
  'share': -2.2497462636110735e-17,
  'symbol': 'cusdt'},
 {'amount': 0.0, 'rewards': 0.0, 'share': 0.0, 'symbol': 'cusdc'}]